In [36]:
import pandas as pd
import datetime
import os
directory = 'data/as_csv'

In [56]:
# Create empty data frame to populate with activities
df_activities_cloud = pd.DataFrame([], columns=['position', 'color'])

#combine all activities into 1 dataset. Drop activities before July 2021--the month I moved to Seattle 
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    df = pd.read_csv(f)
    date_string = df['time'][0][:10]
    date_list = date_string.split('-')
    datetime_obj = datetime.datetime(int(date_list[0]),int(date_list[1]),int(date_list[2]))
    if datetime_obj < datetime.datetime(2021,7,1):
        continue
        
    # Drop time column, and Only keep every 4th row. Data points are dense enough to display adequately with many fewer points.
    df = df.drop(['time'], axis=1)
    df = df.iloc[::2]
    df = df.iloc[::2]
    
    # Set altitude scale to 1.5 to match my custom Mapbox style.
    df['altitude'] = df['altitude']*1.5
    
    df_normalized = (df-df.min())/(df.max()-df.min())
    # convert position to tuples and drop former columns
    df['position'] = df[['latitude', 'longitude', 'altitude']].apply(tuple, axis=1)
    df = df.drop(['latitude', 'longitude', 'altitude'], axis=1)
    
    # add rgb values based on normalized location data
    # df.insert(3, 'r', (df_normalized['latitude']*255).astype(int))
    # df.insert(4, 'g', (df_normalized['longitude']*255).astype(int))
    # df.insert(5, 'b', (df_normalized['altitude']*255).astype(int))

    r = (df_normalized['latitude']*255).astype(int)
    g = (df_normalized['longitude']*255).astype(int)
    b = (df_normalized['altitude']*255).astype(int)

    df.insert(1, 'color', list(zip(r,g,b)))
    

    # Add activity coordinates to the activity
    df_activities_cloud = pd.concat([df_activities_cloud, df])    

In [57]:
df_activities_cloud.to_json('data/as_json/activites_cloud.json', orient='records', lines=True)